<a href="https://colab.research.google.com/github/em-bst/enhancing-large-language-models-with-human-feedback/blob/main/RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trl
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### Reward model training configuration

In [ ]:
""" Full training:
    --model_name_or_path gpt2 \
    --dataset_name trl-lib/ultrafeedback_binarized \
    --output_dir ./drive/reward_output \
    --per_device_train_batch_size 4 \
    --num_train_epochs 1 \
    --learning_rate 1.0e-5 \
    --logging_steps 25 \
    --eval_strategy steps \
    --eval_steps 50 \
    --max_length 1024 """

' Full training:\npython examples/scripts/reward_modeling.py     --model_name_or_path gpt2     --dataset_name trl-lib/ultrafeedback_binarized     --output_dir ./drive/reward_output     --per_device_train_batch_size 4     --num_train_epochs 1     --gradient_checkpointing True     --learning_rate 1.0e-5     --logging_steps 25     --eval_strategy steps     --eval_steps 50     --max_length 2048 '

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


#### Loading of the libraries:

In [3]:
import warnings
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPT2Tokenizer, GenerationConfig, AutoModelForCausalLM
from datasets import load_dataset, DatasetDict, load_from_disk

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    setup_chat_format,
    AutoModelForCausalLMWithValueHead,
    PPOConfig,
    PPOTrainer
)

#### GPUs' Configuration

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#### Configuration of the reward model

In [5]:
model_name_ = "gpt2"
output_dir = "./drive/reward_output"
dataset_name = "trl-lib/ultrafeedback_binarized"


In [6]:
model_args = ModelConfig(model_name_or_path=model_name_)


In [7]:
model_args

ModelConfig(model_name_or_path='gpt2', model_revision='main', torch_dtype=None, trust_remote_code=False, attn_implementation=None, use_peft=False, lora_r=16, lora_alpha=32, lora_dropout=0.05, lora_target_modules=None, lora_modules_to_save=None, lora_task_type='CAUSAL_LM', use_rslora=False, load_in_8bit=False, load_in_4bit=False, bnb_4bit_quant_type='nf4', use_bnb_nested_quant=False)

In [11]:
# Definition of the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Definition of the Model
model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path, num_labels=1, trust_remote_code=model_args.trust_remote_code)
model.config.pad_token_id = tokenizer.pad_token_id
model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=1, bias=False)
)

#### Loading of the dataset : ultrafeedback_binarized from the trl library

In [9]:
dataset = load_dataset("trl-lib/ultrafeedback_binarized")
print(dataset)

README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1000
    })
})


#### Data Preparation ie. tokenization of the data.
The goal is to tokenize the "chosen" and "rejected" text columns and add the resulting tokenized information as new columns (input_ids_chosen, attention_mask_chosen, input_ids_rejected, attention_mask_rejected) to the dataset. These tokenized columns are crucial for training, as they allow the model to process the text inputs efficiently in the training steps.

In [10]:
def tokenize(batch):
    """
    Tokenizes the 'chosen' and 'rejected' columns of a batch of examples.

    This function takes a batch of examples containing the 'chosen' and 'rejected' columns,
    and transforms them into token IDs and attention masks for each text, ready to be used by a model.

    Args:
        batch (dict): A dictionary containing a batch of examples with the following keys:
            - 'chosen': List of texts chosen for each example.
            - 'rejected': List of rejected texts for each example.

    Returns:
        dict: A dictionary containing the following columns for each example:
            - 'input_ids_chosen': Token IDs for the chosen texts.
            - 'attention_mask_chosen': Attention mask for the chosen texts.
            - 'input_ids_rejected': Token IDs for the rejected texts.
            - 'attention_mask_rejected': Attention mask for the rejected texts.
    """
    return tokenizer(
        [(str(chosen), str(rejected)) for (chosen, rejected) in zip(batch["chosen"], batch["rejected"])],
        truncation=True,  # Truncates when length superior to max_length
        max_length=min(tokenizer.model_max_length, 200),  # Limits the sequence length to 200 tokens (for model settings and memory purpose)
        padding="max_length",  # Pads sequences shorter to max_length
        return_tensors="pt"  # Returns the tokenized data as PyTorch tensors
    )

#Apply the tokenize_function to the dataset, processing examples in batches for efficiency
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/62135 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk('/content/drive/MyDrive/tokenized_dataset')

Saving the dataset (0/5 shards):   0%|          | 0/62135 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

#### Taking a subset of the training data for computation and memory purposes

In [12]:
train_subset_indices = torch.randint(0, len(tokenized_dataset["train"]), size=(int(0.02 * len(tokenized_dataset["train"])),))
train_subset = tokenized_dataset["train"].select(train_subset_indices.tolist())

# Update the dataset with the new subset
tokenized_dataset['train'] = train_subset

The dataset is now made of 1,242 training points and 1,000 testing ones.

#### Reward Model Training Parameters 🚀

In [13]:
if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)

script_args = ScriptArguments(
    dataset_name=dataset_name,
    dataset_train_split="train",
    dataset_test_split="test")

training_args = RewardConfig(
    output_dir="/content/drive/MyDrive/reward_output",
    per_device_train_batch_size=8,               # Batch size
    num_train_epochs=1,                          # Number of training epochs
    learning_rate=1.0e-5,                        # Learning rate for training
    logging_steps=25,                            # Interval (in steps) to log training progress
    evaluation_strategy="steps",                 # Evaluates the model at regular steps
    eval_steps=50,
    report_to = "none",                          # Disabling reports to external systems
    dataset_num_proc=4                           # For parallel computing
)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#### Reward Model Training⚡

In [ ]:
##########
# Training
##########
trainer = RewardTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset[script_args.dataset_train_split],
    eval_dataset=tokenized_dataset[script_args.dataset_test_split],
    )
trainer.train()

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1242 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1210 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1074 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1054 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 1024). Running this sequence through the model will result in indexing errors


Filter (num_proc=4):   0%|          | 0/1242 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
50,0.819300,0.693147,0.508824


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>user                              │ <|im_start|>user                             │ [0.4218, 0.5782] │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improve the operational performance of its   │                  │
│ reduce the number and severity of traffic     │ transportation network;   reduce the number  │                  │
│ collisions and increase driver, passenger,    │ and severity of traffic collisions and       │                  │
│ and pedestrian safety;  collect, disseminate, │ increase driver, passenger, and pedestrian   │                  │
│ and utilize real-time traffic, transit,       │ safety;  collect, disseminate, and utilize   │                  │
│ parking, and other transportation-related     │ real-time traffic, transit, parking, and     │                  │
│ information to improve mobility, reduce       │ other 

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:785: UserWarning: There are 210 out of 550 instances where the predictions for both options are equal. These instances are ignored in the accuracy computation.
  warnings.warn(


TrainOutput(global_step=86, training_loss=0.8161915956541549, metrics={'train_runtime': 165.1817, 'train_samples_per_second': 4.153, 'train_steps_per_second': 0.521, 'total_flos': 0.0, 'train_loss': 0.8161915956541549, 'epoch': 1.0})

#### Saving the training parameters and the model

In [ ]:
trainer.save_model(training_args.output_dir) #Saving the training parameters
model.save_pretrained("drive/MyDrive/reward_output/model") #Saving the model

### Fine-tuning a language model via  Proximal Policy Optimization (PPO)

#### Loading pretrained model, reference model and value model

Instead of using AutoModelForCausalLMWithValueHead as it was done in https://huggingface.co/docs/trl/quickstart#minimal-examplehttps://huggingface.co/docs/trl/quickstart#minimal-example
, we differentiate by explicitly using a separate model for value estimation. This involves using:

- AutoModelForCausalLM for the text generation model model_with_ppo which is responsible for generating text.
- AutoModelForSequenceClassification for the value_model, which is responsible for estimating the value of the generated sequence.

In [15]:
model_with_ppo = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
ref_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
value_model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=1).to(device)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### PPO Reward Model Training Parameters 🚀

In [16]:
ppo_config = {"mini_batch_size": 1,
              "batch_size": 1,
              "num_train_epochs" : 1,
              "num_ppo_epochs": 1,
              "output_dir" : "/content/drive/MyDrive/ppo_output",
              "learning_rate": 1.0e-5,
              "kl_coef": 0.05,
              "logging_steps" : 25,
              "evaluation_strategy" : "steps",
              "eval_steps" : 50,
              "report_to": "none",
              "dataset_num_proc": 4,
              "reward_model_path" : "/content/drive/MyDrive/reward_output"
              }

config = PPOConfig(**ppo_config)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#### PPO Reward Model Training⚡

In [17]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [18]:
ppo_trainer = PPOTrainer(config,
                         processing_class=tokenizer,
                         model = model_with_ppo,
                         ref_model = ref_model,
                         value_model = value_model,
                         reward_model = model,
                         train_dataset = tokenized_dataset["train"],
                         eval_dataset = tokenized_dataset["test"]
                         )

In [22]:
train_ppo_results = ppo_trainer.train()
train_ppo_results

===training policy===


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:630: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  metrics["val/ratio_var"] = self.accelerator.gather(ratio_stats).var().item()


Step,Training Loss,Validation Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │ [{'content': 'As an HR manager, you want to  │ 5.506343364715576  │
│ test a potential employee\'s ability to     │ test a potential employee\'s ability to      │                    │
│ solve puzzles to determine their            │ solve puzzles to determine their suitability │                    │
│ suitability for a job. Write a Python       │ for a job. Write a Python script that        │                    │
│ script that generates a list of questions   │ generates a list of questions that require   │                    │
│ that require logical reasoning to answer.   │ logical reasoning to answer. Your list       │                    │
│ Your list should include questions related  │ should                                       │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["[{'content': 'As an HR manager, │                                              │                    │
│ you want to test a potential employee\'s    │                                              │                    │
│ ability to solve puzzles to determine their │                                              │                    │
│ suitability for a job. Write a Python       │                                              │                    │
│ script that generates a list of questions   │                                              │                    │
│ that require logical reasoning to answer.   │                                              │                    │
│ Your list should include questions related  │                                              │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["                                │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ [{'content': 'Definition: In this task, you │ .                                            │ 6.458003044128418  │
│ are given a summary for US Congressional    │                                              │                    │
│ and California state bill, your task is to  │ The following table shows the number of      │                    │
│ generate a Title for this bill. The         │ projects that have been funded by the        │                    │
│ preferred titles are under forty words and  │ Department of Transportation (DOT) in the    │                    │
│ mention the purpose of the bill.\nInput:    │ last fis

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:630: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  metrics["val/ratio_var"] = self.accelerator.gather(ratio_stats).var().item()


...

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:630: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  metrics["val/ratio_var"] = self.accelerator.gather(ratio_stats).var().item()


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │ [{'content': 'As an HR manager, you want to  │ 5.506343364715576  │
│ test a potential employee\'s ability to     │ test a potential employee\'s ability to      │                    │
│ solve puzzles to determine their            │ solve puzzles to determine their suitability │                    │
│ suitability for a job. Write a Python       │ for a job. Write a Python script that        │                    │
│ script that generates a list of questions   │ generates a list of questions that require   │                    │
│ that require logical reasoning to answer.   │ logical reasoning to answer. Your list       │                    │
│ Your list should include questions related  │ should                                       │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["[{'content': 'As an HR manager, │                                              │                    │
│ you want to test a potential employee\'s    │                                              │                    │
│ ability to solve puzzles to determine their │                                              │                    │
│ suitability for a job. Write a Python       │                                              │                    │
│ script that generates a list of questions   │                                              │                    │
│ that require logical reasoning to answer.   │                                              │                    │
│ Your list should include questions related  │                                              │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["                                │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ [{'content': 'Definition: In this task, you │ .                                            │ 6.458003044128418  │
│ are given a summary for US Congressional    │                                              │                    │
│ and California state bill, your task is to  │ The following table shows the number of      │                    │
│ generate a Title for this bill. The         │ projects that have been funded by the        │                    │
│ preferred titles are under forty words and  │ Department of Transportation (DOT) in the    │                    │
│ mention the purpose of the bill.\nInput:    │ last fis

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:630: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  metrics["val/ratio_var"] = self.accelerator.gather(ratio_stats).var().item()


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │ [{'content': 'As an HR manager, you want to  │ 5.506343364715576  │
│ test a potential employee\'s ability to     │ test a potential employee\'s ability to      │                    │
│ solve puzzles to determine their            │ solve puzzles to determine their suitability │                    │
│ suitability for a job. Write a Python       │ for a job. Write a Python script that        │                    │
│ script that generates a list of questions   │ generates a list of questions that require   │                    │
│ that require logical reasoning to answer.   │ logical reasoning to answer. Your list       │                    │
│ Your list should include questions related  │ should                                       │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["[{'content': 'As an HR manager, │                                              │                    │
│ you want to test a potential employee\'s    │                                              │                    │
│ ability to solve puzzles to determine their │                                              │                    │
│ suitability for a job. Write a Python       │                                              │                    │
│ script that generates a list of questions   │                                              │                    │
│ that require logical reasoning to answer.   │                                              │                    │
│ Your list should include questions related  │                                              │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["                                │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ [{'content': 'Definition: In this task, you │ .                                            │ 6.458003044128418  │
│ are given a summary for US Congressional    │                                              │                    │
│ and California state bill, your task is to  │ The following table shows the number of      │                    │
│ generate a Title for this bill. The         │ projects that have been funded by the        │                    │
│ preferred titles are under forty words and  │ Department of Transportation (DOT) in the    │                    │
│ mention the purpose of the bill.\nInput:    │ last fis

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:630: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  metrics["val/ratio_var"] = self.accelerator.gather(ratio_stats).var().item()


In [20]:
model_with_ppo.save_pretrained("drive/MyDrive/reward_output/model_with_ppo")

#### Performance of the PPO Reward Model : Mean Reward and Objective KL Divergence metrics

Unfortunately, after trying to run it several time, this could not run in its entirety due to CUDA out of memory.

In [35]:
import torch.nn.functional as F

def collect_metrics(batch, ppo_trainer, original_dataset):
    """
    Collects and calculates metrics for evaluating the performance of the PPO trainer : mean reward and Objective KL Divergence.
    Source : https://huggingface.co/docs/trl/how_to_train

    Args:
        batch : A batch of data used for evaluation.
        ppo_trainer : The PPO trainer instance.
        original_dataset : The original dataset containing 'chosen' and 'rejected' columns.

    Returns:
        A tuple containing the mean reward and KL divergence.
    """
    original_batch = original_dataset.select(range(batch['input_ids'].shape[0]))

    # Use the reward model directly to compute rewards
    reward_inputs = tokenizer([(str(chosen), str(rejected)) for (chosen, rejected) in zip(original_batch["chosen"], original_batch["rejected"])],
                             truncation=True,
                             max_length=min(tokenizer.model_max_length, 200),
                             padding="max_length",
                             return_tensors="pt")

    # Move reward inputs to the appropriate device
    reward_inputs = {k: v.to(ppo_trainer.reward_model.device) for k, v in reward_inputs.items()}

    rewards = ppo_trainer.reward_model(**reward_inputs).logits[:, 0].detach().cpu().numpy()

    mean_reward = rewards.mean()

    model_logits = ppo_trainer.model(batch['input_ids'].to(ppo_trainer.model.device))['logits']
    ref_model_logits = ppo_trainer.ref_model(batch['input_ids'].to(ppo_trainer.ref_model.device))['logits']

    kl_divergence = F.kl_div(
        F.log_softmax(model_logits, dim=-1),
        F.softmax(ref_model_logits, dim=-1),
        reduction='batchmean'
    ).item()

    return mean_reward, kl_divergence

In [36]:
collect_metrics(tokenized_dataset["train"][0], ppo_trainer, tokenized_dataset["train"])

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

OutOfMemoryError: CUDA out of memory. Tried to allocate 352.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 201.06 MiB is free. Process 2692 has 14.55 GiB memory in use. Of the allocated memory 13.79 GiB is allocated by PyTorch, and 641.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#### Generation of a sample output from the optimized model:

Source for this code : https://huggingface.co/docs/trl/quickstart#minimal-example

First Example :

In [60]:
# Step 1: Encode the query
query_txt = "This morning I went to the "  # Your input query
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model_with_ppo.device)

# Step 2: Define generation parameters
generation_kwargs = {
    "min_length": -1,
    "top_k": 0,        # Disable top-k sampling
    "top_p": 1.0,      # Enable nucleus sampling
    "do_sample": True, # Activate sampling
    "pad_token_id": tokenizer.eos_token_id,  # Padding token ID
    "max_new_tokens": 20,  # Maximum length of the generated response
}

# Step 3: Generate model response
response_tensor = model_with_ppo.generate(query_tensor, **generation_kwargs)

# Step 4: Decode the response
response_txt = tokenizer.decode(response_tensor[0])

# Display the generated response
print("Query:", query_txt)
print("Generated Response:", response_txt)


Query: This morning I went to the 
Generated Response: This morning I went to the  mile-long Air Base Towers where there are the buildings slightly above ground level and a trailer base


Other examples :

In [54]:
# Defining diverse queries
queries = [
    "This morning I went to the ",
    "The best way to learn something new is ",
    "A perfect vacation destination could be ",
    "For dinner tonight, I am planning to make ",
    "One of the most fascinating historical events is ",
    "The importance of exercise in daily life is ",
    "My favorite childhood memory is ",
    "The technology of the future might include ",
    "A great way to spend a weekend is ",
    "The secret to happiness might be found in "
]

# Loop through each query and run the steps defined above
for query_txt in queries:
    # Step 1: Encode the query
    query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model_with_ppo.device)

    # Step 2: Generate model response
    generation_kwargs = {
        "min_length": -1,
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
        "max_new_tokens": 20,
    }

    # Step 3: Generate model response
    response_tensor = model_with_ppo.generate(query_tensor, **generation_kwargs)

    # Step 4: Decode the response
    response_txt = tokenizer.decode(response_tensor[0])


    # Display the generated response
    print(f"Query: {query_txt}")
    print(f"Generated Response: {response_txt}")


Query: This morning I went to the 
Generated Response: This morning I went to the 【M trou laity Centre】 and took advantage of the free demonstrations and news as well as the
Query: The best way to learn something new is 
Generated Response: The best way to learn something new is  to start with the basics. If you realise that something you have solved, you know that this
Query: A perfect vacation destination could be 
Generated Response: A perfect vacation destination could be iced just right... by having no THC that is produced by used-fueled tourism. That total
Query: For dinner tonight, I am planning to make 
Generated Response: For dinner tonight, I am planning to make iced tea with some ground walnuts in a lacy crispy dough. My husband and I know how
Query: One of the most fascinating historical events is 
Generated Response: One of the most fascinating historical events is  The Reign of Whytli .  The story begins shortly after the fall of the Roman
Query: The importance of exercis

#### Saving the generated outputs from the optimized LLM:

In [65]:
file_path = "/content/drive/My Drive/generated-outputs-ppo.txt"

with open(file_path, "a") as file:
    for query_txt in queries:
        # Step 1: Encode the query
        query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model_with_ppo.device)

        # Step 2: Define generation parameters
        generation_kwargs = {
            "min_length": -1,
            "top_k": 0.0,
            "top_p": 1.0,
            "do_sample": True,
            "pad_token_id": tokenizer.eos_token_id,
            "max_new_tokens": 30,
        }

        # Step 3: Generate model response
        response_tensor = model_with_ppo.generate(query_tensor, **generation_kwargs)

        # Step 4: Decode the response
        response_txt = tokenizer.decode(response_tensor[0], skip_special_tokens=True)

        # Write the query and response to the file
        file.write(f"Query: {query_txt}\n")
        file.write(f"Generated Response: {response_txt}\n")
        file.write("\n" + "-"*50 + "\n")  # Add separator between queries